# <font style="font-family:roboto;color:#455e6c"> Parametrising a Machine Learning Interatomic Potential </font>  

<div class="admonition note" name="html-admonition" style="background:#e3f2fd; padding: 10px">
<font style="font-family:roboto;color:#455e6c"> <b> DPG Tutorial: Automated Workflows and Machine Learning for Materials Science Simulations </b> </font> </br>
<font style="font-family:roboto;color:#455e6c"> 16 March 2025 </font> </br> </br>
Marvin Poul, Sarath Menon, Haitham Gaafer, Jörg Neugebauer </br>
<i> Max Planck Institute for Sustainable Materials </i></br>
</br>
Minaam Qamar, Ralf Drautz </br>
<i> Ruhr-Universität Bochum </i></br>
</br>
Tilmann Hickel </br>
<i> Bundesanstalt für Materialforschung und -prüfung </i></br>
</div>

In this notebook we fit an [Atomic Cluster Expansion](https://doi.org/10.1103/PhysRevB.99.014104) interatomic potential using the [pacemaker](https://www.nature.com/articles/s41524-021-00559-9) software.

In [ ]:
from pyironflow.pyironflow import PyironFlow
import matplotlib.pyplot as plt
from pyiron_nodes.atomistic.mlips.fitting.ace import *

## <font style="font-family:roboto;color:#455e6c"> Loading the dataset </font> 

Recalling the workflow, we are in the first essential step of loading the training dataset

<img src="img/highlighted_workflow.png" width="50%">

As a first step, we load the dataset by specifying the `file_path'. This dataset has been generated for the CaMg system using the ASSYST approach discussed in the previous session [02_assyst.ipynb](02_assyst.ipynb).

In [ ]:
load_dataset = ReadPickledDatasetAsDataframe(file_path = "data/mgca.pckl.tgz", compression = None)
load_dataset.pull();

The histogram of the total energies of all atomic structures in the dataset is plotted using the `PlotEnergyHistogram' node.

In [ ]:
hist_plot = PlotEnergyHistogram(df = load_dataset.outputs.df, log_scale = False)
hist_plot.pull();

Similarly, using the `PlotForcesHistogram` Node, we can plot the forces histogram

In [ ]:
hist_plot = PlotForcesHistogram(df = load_dataset.outputs.df, log_scale = True)
hist_plot.pull();

## <font style="font-family:roboto;color:#455e6c"> Split the dataset into training and test </font> 

In a second step, we split the dataset into training and testing datasets.

This is done by choosing the percentage used for the training dataset through the `training_frac` parameter, where `training_frac = 0.5` means we use 50% for training and 50% for testing.

In [ ]:
split_dataset = SplitTrainingAndTesting(data_df = load_dataset.outputs.df, training_frac = 0.5)

## <font style="font-family:roboto;color:#455e6c"> Define and specify the configuration of the ACE potential </font> 

Following the `PACEmaker` notation, we create a file similar to `input.yaml` (see the `PACEmaker` [documentation](https://pacemaker.readthedocs.io/en/latest/) for more details).

```
fit:
  fit_cycles: 1
  loss: {L1_coeffs: 1e-08, L2_coeffs: 1e-08, kappa: 0.08, w0_rad: 0, w1_rad: 0, w2_rad: 0}
  maxiter: 1000
  optimizer: BFGS
  trainable_parameters: ALL
  weighting: {DE: 1.0, DElow: 1.0, DEup: 10.0, DF: 1.0, DFup: 50.0, energy: convex_hull,
    nfit: 20000, reftype: all, seed: 42, type: EnergyBasedWeightingPolicy, wlow: 0.95}
.
.
.
potential:
  bonds:
    ALL:
      dcut: 0.01
      radbase: SBessel
      radparameters: [5.25]
      rcut: 6.0
  elements: [Mg, Ca]
  embeddings:
    ALL:
      fs_parameters: [1, 1, 1, 0.5]
      ndensity: 2
      npot: FinnisSinclairShiftedScaled
  functions:
    number_of_functions_per_element = 300
    ALL:
      lmax_by_orders: [15, 6, 2, 1]
      nradmax_by_orders: [0, 6, 3, 1]

### 1. Embeddings
specify how the atomic energy $E_i$ depends on the ACE properties/densities $\varphi$. The most approximate approach, but the most efficient for potential fitting, is the linear expansion $E_i = \varphi$. Non-linear expansions, e.g. including the square root, provide more flexibility and accuracy of the final potential, but require significantly more computational resources for fitting.

Embeddings for `ALL` species: 
- non-linear `FinnisSinclairShiftedScaled`
- 2 densities
- fs_parameters': [1, 1, 1, 0.5]:
$$E_i = 1.0 * \varphi(1)^1 + 1.0 * \varphi(2)^{0.5} = \varphi^{(1)} + \sqrt{\varphi^{(2)}} $$

### 2. Radial functions

Radial functions are defined by orthogonal polynomals. Examples are:
* (a) Exponentially-scaled Chebyshev polynomials (λ = 5.25)
* (b) Power-law scaled Chebyshev polynomials (λ = 2.0)
* (c) Simplified spherical Bessel functions

<img src="img/radial-functions-low.png" width="40%">

Radial functions specification have to be provided for `ALL` species pairs (i.e. Al-Al, Al-Li, Li-Al, Li-Li):

* based on the Simplified Bessel function
* and cutoff, e.g. $r_c=6.0$

#### 3. B-basis functions

B-basis functions  specifications for `ALL` species type interactions, i.e. Al-Al block:
* maximum order = 4, i.e. body-order 5 (1 central atom + 4 neighbour  densities)
* nradmax_by_orders: 15, 3, 2, 1
* lmax_by_orders: 0, 3, 2, 1

For simplicity, the main inputs that we will consider for the potential configurations are:

- `number_of_functions_per_element`: specifies how many functions will be provided in the potential
- `rcut`: specifies what the cutoff radius is

In [ ]:
parameterize_potential = ParameterizePotentialConfig(number_of_functions = 10, rcut = 6.0)

Check the current potential configurations in dictionary format,

In [ ]:
parameterize_potential.pull().to_dict()

## <font style="font-family:roboto;color:#455e6c"> Linear fitting </font>

Finally, we run our fit with the thus defined `potential_config` and then save the potential files inside a new folder.

**Note:** Setting `verbose = True` will show all the details of building the design matrices.

In [ ]:
run_linear_fit = RunLinearFit(potential_config = parameterize_potential,
                                                df_train = split_dataset.outputs.df_training,
                                                df_test= split_dataset.outputs.df_testing, verbose = False)

In [ ]:
save_potential = SavePotential(basis = run_linear_fit.outputs.basis)

In [ ]:
save_potential.pull()

## <font style="font-family:roboto;color:#455e6c"> Workflow </font>

In [ ]:
wf = make_linearfit(workflow_name= 'LinearAceDataset', file_path='data/mgca.pckl.tgz', delete_existing_savefiles=True)

In [ ]:
wf.draw(size=(20,10))

You can save the workflow using `wf.save()` to call it back later without the need to re-run it.

In [ ]:
wf.save()

## <font style="font-family:roboto;color:#455e6c"> Loading the workflow into the GUI </font>

Run the workflow using the GUI and perform a level 1 validation of our linear ACE potential.

Helpful nodes for performing this task:
- `atomistic -> mlips -> fitting -> ace`: contains all nodes to run the linear ACE fit, to plot the data histograms and the fitting's accuracy curves.

<img src="img/validation_schematic.png" width="60%">

**NOTE:** You can change the ratio of the canvas to the whole screen by changing the value of `flow_widget_ratio` between 0 to 1 (try 0.6 or 0.7)

In [ ]:
pf = PyironFlow([wf], flow_widget_ratio = 0.7)
pf.gui

Exercise: change the `number_of_functions_per_element` to a higher value (i.e., 50) and check the fitting curves. Did the fit get better or worse?

**Note:** You can save the potential under a new name using the `filename` input in `save_potential` node.

<img src="img/logo_roll.png" width="1200">